In [3]:
import datetime
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import joblib
 
import azureml.core
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core import Experiment
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage
from azureml.core.webservice import AciWebservice
from azureml.core.conda_dependencies import CondaDependencies

In [4]:
ws = Workspace.create(name='PythonSDKDeploy',
                      subscription_id='6a29f7fb-e4b8-4362-85c5-557beba36561', 
                      resource_group='New',
                      create_resource_group=False,
                      location='central india'
                     )

Deploying StorageAccount with name pythonsdstoragee294be6ae.
Deploying AppInsights with name pythonsdinsights5c9c7c5f.
Deployed AppInsights with name pythonsdinsights5c9c7c5f. Took 5.41 seconds.
Deploying KeyVault with name pythonsdkeyvaultd02212b9.
Deployed KeyVault with name pythonsdkeyvaultd02212b9. Took 19.04 seconds.
Deployed StorageAccount with name pythonsdstoragee294be6ae. Took 22.76 seconds.
Deploying Workspace with name PythonSDKDeploy.
Deployed Workspace with name PythonSDKDeploy. Took 41.65 seconds.


In [5]:
exp = Experiment(workspace=ws, name='deploysdk')
run = exp.start_logging()                   
run.log("Experiment start time", str(datetime.datetime.now()))

In [6]:
model = Model.register(model_path = "save_model/insaurance_LR.sav",
                       model_name = "insurance_model",
                       tags = {"key": "1"},
                       description = "Insurance Prediction",
                       workspace = ws)

Registering model insurance_model


In [7]:
salenv = CondaDependencies()
salenv.add_conda_package("scikit-learn")
 
with open("salenv.yml","w") as f:
    f.write(salenv.serialize_to_string())
with open("salenv.yml","r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - azureml-defaults

- scikit-learn
channels:
- anaconda
- conda-forge



In [8]:
image_config = ContainerImage.image_configuration(execution_script="app.py", 
                                                  runtime="python", 
                                                  conda_file="salenv.yml")

<ipython-input-8-1b2283e29a57>:1: DeprecationWarning: ContainerImage class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
  image_config = ContainerImage.image_configuration(execution_script="app.py",


In [9]:
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "Insurance",  "method" : "sklearn"}, 
                                               description='Predict Insurance Price')
 
service = Webservice.deploy_from_model(workspace=ws,
                                       name='salary-svc',
                                       deployment_config=aciconfig,
                                       models=[model],
                                       image_config=image_config)
 
service.wait_for_deployment(show_output=True)

<ipython-input-9-8dd48779a4fb>:6: DeprecationWarning: deploy_from_model has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
  service = Webservice.deploy_from_model(workspace=ws,


Creating image
Running................................................................................
Succeeded
Image creation operation finished for image salary-svc:1, operation "Succeeded"
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..................................................
Failed


Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 8cd05b59-88d3-4fc1-9309-bb425e1af279
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: salary-svc. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can also try to run image c826fdaf488a43f294d50c8d85c87179.azurecr.io/salary-svc@sha256:56ac115e319893a40e25eb6d540f91deadf3b57c503ce903b3e82da6ee54bd05 locally. Please refer to https://aka.ms/debugimage#service-launch-fails for more information.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlea

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 8cd05b59-88d3-4fc1-9309-bb425e1af279
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: salary-svc. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can also try to run image c826fdaf488a43f294d50c8d85c87179.azurecr.io/salary-svc@sha256:56ac115e319893a40e25eb6d540f91deadf3b57c503ce903b3e82da6ee54bd05 locally. Please refer to https://aka.ms/debugimage#service-launch-fails for more information.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Your container application crashed. This may be caused by 

In [10]:
print(service.scoring_uri)

None
